Primeiro conseguir separar a imagem por regiao onde identifica a prova e onde marca as questoes

In [1]:

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd


#Diretório da pasta que contém as imagens
diretorio_pasta = '.\Leitor\provas_realizadas'

# Lista de arquivos na pasta
arquivos_na_pasta = os.listdir(diretorio_pasta)
index = 0

for arquivo in arquivos_na_pasta:
    if arquivo.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        try:
            caminho_arquivo = os.path.join(diretorio_pasta, arquivo)

            respostas_A = ["E","E","C", "C","C","E","E","E","C","C", "C","E","C","C", "C","E","C","E","C","E","C", "A","C","B","E","A","E","D", "A","C","C","C", "B", "C", 
                        "C","E","C", "E","A","A","E","B","C","E", "E","B","C","E","NULL","E","C", "E","E","C","C","D","C","E", "E","C"]


            respostas_B = ["C","A","C", "A","C","E","E","E","C","E", "A","E","C","C", "C","E","C","E","C","E","E", "A","C","B","E","E","E","D", "C","C","C","C", "B", "C", 
                        "C","E","C", "E","C","A","E","B","C","C", "E","B","C","E","NULL","E","C", "E","E","C","C","D","C","E", "E","C"]


            respostas_C = ["C","C","E", "B","C","C","E","E","C","E", "A","E","C","C", "C","E","E","E","C","E","E", "A","C","B","E","E","C","D", "C","C","C","C", "B", "C", 
                        "C","E","C", "E","C","A","E","B","C","C", "E","B","C","E","NULL","E","C", "E","E","C","C","D","C","E", "E","C"]

            respostas_D = ["C","C","E", "B","C","C","E","E","C","E", "A","E","C","C", "C","E","E","E","C","E","E", "A","C","B","E","E","C","D", "C","C","C","C", "B", "C", 
                        "C","E","C", "E","C","A","E","B","C","C", "E","B","C","E","NULL","E","C", "E","E","C","C","D","C","E", "E","C"]

            respostas_E = ["C","C","E", "B","C","C","E","E","C","E", "A","E","C","C", "C","E","E","E","C","E","E", "A","C","B","E","E","C","D", "C","C","C","C", "B", "C", 
                        "C","E","C", "E","C","A","E","B","C","C", "E","B","C","E","NULL","E","C", "E","E","C","C","D","C","E", "E","C"]

            image = cv2.imread(caminho_arquivo)
            image_resize = cv2.resize(image, None, fx=0.5, fy=0.5)
            image_copy = cv2.resize(image, None, fx=0.5, fy=0.5)

            # Converta a imagem para tons de cinza
            gray = cv2.cvtColor(image_copy, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray,(1,1),1000)
            _, thresh = cv2.threshold(blur, 195, 255, cv2.THRESH_BINARY)

            # Detecte os contornos na imagem
            contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            contours = sorted(contours, key=cv2.contourArea, reverse=True) 

            area_minima_desejada = 10
            area_maxima_desejada = 60000

            # Filtrar os contornos com área maior que o valor desejado
            filtered_countourns = [cont for cont in contours if (cv2.contourArea(cont) > area_minima_desejada) and (cv2.contourArea(cont) < area_maxima_desejada)]

            cv2.drawContours(thresh, filtered_countourns, -1, (0, 0, 255), 1)

            nova_imagem = np.zeros_like(thresh)

            # Desenhe os contornos filtrados na nova imagem
            cv2.drawContours(nova_imagem, filtered_countourns, -1, (255, 255, 255), thickness=cv2.FILLED)

            # Salve a nova imagem
            #cv2.imshow('nova_imagem.png', nova_imagem)

            maior_contorno = max(filtered_countourns, key=cv2.contourArea)

            # Encontre o ângulo de rotação com base na orientação do maior contorno
            (x, y), (largura, altura), angulo = cv2.minAreaRect(maior_contorno)

            # Garanta que o ângulo esteja no intervalo de -45 a 45 graus
            if angulo < -45:
                angulo += 90

            # Rotação da imagem para endireitar
            altura, largura = nova_imagem.shape[:2]
            rotacao_matriz = cv2.getRotationMatrix2D((largura / 2, altura / 2), angulo, 1)
            imagem_endireitada = cv2.warpAffine(nova_imagem, rotacao_matriz, (largura, altura), flags=cv2.INTER_LINEAR)

            # Mostre a imagem endireitada
            #cv2.imshow('Imagem Endireitada.png', imagem_endireitada)

            filtro_nitidez = np.array([[-1, -1, -1],
                                    [-1,  9, -1],
                                    [-1, -1, -1]])
            imagem_nitidez = cv2.filter2D(imagem_endireitada, -1, filtro_nitidez)

            _, imagem_binaria = cv2.threshold(imagem_nitidez, 128, 255, cv2.THRESH_BINARY)

            contornos, _ = cv2.findContours(imagem_nitidez, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Lista para armazenar os retângulos que atendem aos critérios
            retangulos_desejados = []

            # Encontre e desenhe as áreas retangulares que atendem aos critérios
            for contorno in contornos:
                x, y, largura, altura = cv2.boundingRect(contorno)
                
                if largura >= 30 * altura:
                    # Adicione as coordenadas do retângulo desejado à lista
                    retangulos_desejados.append((x, y, largura, altura))

            # Recorte as áreas dos retângulos desejados da imagem original
            areas_recortadas = []
            for x, y, largura, altura in retangulos_desejados:
                area_recortada = imagem_nitidez[y:y+altura, x:x+largura]
                areas_recortadas.append(area_recortada)

            altura, largura = area_recortada.shape[:2]
            comprimento_parte = largura // 5

            prova = 0
            partes_imagem = []
            for i in range(5):
                parte = area_recortada[:, i * comprimento_parte: (i + 1) * comprimento_parte]
                partes_imagem.append(parte)

            # Detecte círculos em cada parte da imagem
            for i, parte in enumerate(partes_imagem):
                altura1, largura1 = parte.shape
                regiao_interesse = None
                final = False
            # Defina a largura da região de interesse como os primeiros 25% da imagem
                match i:
                    case 0:
                        print('entrou 1')
                        largura_regiao_interesse = int(largura1 * 0.20)
                        regiao_interesse = parte[:, :largura_regiao_interesse]
                    case 1:
                        print('entrou 2')
                        largura_regiao_interesse = int(largura1 * 0.30)
                        regiao_interesse = parte[:, :largura_regiao_interesse]
                    case 2:
                        print('entrou 3')
                        largura_regiao_interesse = int(largura1 * 0.40)
                        regiao_interesse = parte[:, :largura_regiao_interesse]
                    case 3:
                        print('entrou 4')
                        largura_regiao_interesse = int(largura1 * 0.45)
                        regiao_interesse = parte[:, :largura_regiao_interesse]
                    case 4:
                        print('entrou 5')
                        largura_regiao_interesse = int(largura1 * 0.65)
                        regiao_interesse = parte[:, :largura_regiao_interesse]
                # Selecione a região de interesse na imagem
                #cv2.imshow(f'Região de Interesse {i}', regiao_interesse)

                circulos = cv2.HoughCircles(
                    regiao_interesse,
                    cv2.HOUGH_GRADIENT,
                    dp=1,
                    minDist=12,  # Distância mínima entre os centros dos círculos
                    param1=6,   # Limiar para detecção de bordas
                    param2=6,   # Limiar para detecção de círculos
                    minRadius=4,  # Raio mínimo do círculo
                    maxRadius=8   # Raio máximo do círculo
                )

                if circulos is not None:
                    circulos = np.uint16(np.around(circulos))

                    # Itere pelos círculos detectados
                    for circulo in circulos[0, :]:
                        centro_x, centro_y, raio = circulo
                        intensidade_cor_centro = None

                        if centro_x >= raio and centro_x < regiao_interesse.shape[1] - raio and centro_y >= raio and centro_y < regiao_interesse.shape[0] - raio:
                            intensidade_cor_centro = regiao_interesse[centro_y, centro_x]
                            cv2.circle(regiao_interesse, (centro_x, centro_y), 2, -1)
                            cv2.imshow(f'{i}', regiao_interesse)
                        if(intensidade_cor_centro != None):
                            print(intensidade_cor_centro)
                            #Determine se a intensidade de cor no centro do círculo é próxima ao branco (intensidade alta)
                            if intensidade_cor_centro > 250:
                                prova = i + 1
                                final = True
                                break
                if(final):
                    break
            # print(f'{index}° prova')
            print(prova)
            # Exemplo de classificação final
            if prova == 1:
                print('A imagem é classificada como A')
            elif prova == 2:
                print('A imagem é classificada como B')
            elif prova == 3:
                print('A imagem é classificada como C')
            elif prova == 4:
                print('A imagem é classificada como D')
            elif prova == 5:
                print('A imagem é classificada como E')

            # Exiba a imagem com as áreas retangulares desenhadas (opcional)
            #cv2.imshow('Imagem com Áreas Retangulares', imagem_nitidez)

            retangulos_desejados1 = []

            imagem_cont = []
            nova_img = np.zeros_like(imagem_nitidez)
            for i, contorno in enumerate(contornos):
                # Crie uma máscara preta do mesmo tamanho da imagem original
                mask = np.zeros_like(gray)

                # Desenhe o contorno atual na máscara (branco)
                cv2.drawContours(mask, [contorno], 0, 255, thickness=cv2.FILLED)

                # Aplique a máscara à imagem original para obter a imagem do contorno atual
                contorno_imagem = cv2.bitwise_and(imagem_nitidez, imagem_nitidez, mask=mask)
                # Mostre ou salve a imagem do contorno atual
                if i in range(1, 6): 
                    imagem_cont.append(contorno_imagem)

            imagem_combinada = np.zeros_like(imagem_nitidez)
            for imagem in imagem_cont:
                imagem_combinada = cv2.add(imagem_combinada, imagem)

            altura, largura = imagem_combinada.shape[:2]
            comprimento_parte = largura // 4

            contornos_combinada, _ = cv2.findContours(imagem_combinada, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            y_max = 0
            x_total = 0
            yi = 600
            xi = 600
            index = 0

            for cont in contornos_combinada: 
            # Crie uma nova imagem com a altura e largura do contorno da imagem combinada
                x, y, largura, altura = cv2.boundingRect(cont)
                xi = min(xi, x)
                yi = min(yi, y)
                x_total += largura
                y_max = max(y_max, y + altura)
            print(xi, yi)
            image_prova = imagem_combinada[yi:y_max, xi:x_total]
            #cv2.imshow('imagem', image_prova)

            partes_imagem2 = []

            altura, largura = image_prova.shape
            parte1_x1, parte1_y1, parte1_x2, parte1_y2 = 0, 0, largura // 4, altura
            parte2_x1, parte2_y1, parte2_x2, parte2_y2 = largura // 4, 0, largura // 2, altura
            parte3_x1, parte3_y1, parte3_x2, parte3_y2 = largura // 2, 0, 3 * largura // 4, altura

            # Recorte as três partes
            parte1 = image_prova[parte1_y1:parte1_y2, parte1_x1:parte1_x2]
            parte2 = image_prova[parte2_y1:parte2_y2, parte2_x1:parte2_x2]
            parte3 = image_prova[parte3_y1:parte3_y2, parte3_x1:parte3_x2]

            partes_imagem2.append(parte1)
            partes_imagem2.append(parte2)
            partes_imagem2.append(parte3)

            #cv2.imshow(f'img combinada 1', parte1)
            #cv2.imshow(f'img combinada 2', parte2)
            #cv2.imshow(f'img combinada 3', parte3)
            respostas = []

            partes_imagem3 = []
            altura_parte = altura // 25
            for j in range (0,3):
                for i in range (25):
                    if(j == 2 and i == 10):
                        break
                    calc = (i + 1) * altura_parte + 1
                    parte3 = partes_imagem2[j][i * altura_parte: calc, :]
                    partes_imagem3.append(parte3)
                    #cv2.imwrite(f'{j} - {i}.png', parte3)
                    largura = parte3.shape[1]
                    altura = parte3.shape[0]

                    # Defina as faixas de porcentagem
                    faixa_A = (0.5, 0.61)
                    faixa_B = (0.61, 0.72)
                    faixa_C = (0.72, 0.83)
                    faixa_D = (0.83, 0.925)
                    faixa_E = (0.925, 1)

                    # Determine a ROI com base nas faixas de porcentagem
                    x1_A, x2_A = int(largura * faixa_A[0]), int(largura * faixa_A[1])
                    x1_B, x2_B = int(largura * faixa_B[0]), int(largura * faixa_B[1])
                    x1_C, x2_C = int(largura * faixa_C[0]), int(largura * faixa_C[1])
                    x1_D, x2_D = int(largura * faixa_D[0]), int(largura * faixa_D[1])
                    x1_E, x2_E = int(largura * faixa_E[0]), int(largura * faixa_E[1])

                    # Recorte as ROIs com base nas coordenadas calculadas
                    roi_A = parte3[:, x1_A:x2_A]
                    roi_B = parte3[:, x1_B:x2_B]
                    roi_C = parte3[:, x1_C:x2_C]
                    roi_D = parte3[:, x1_D:x2_D]
                    roi_E = parte3[:, x1_E:x2_E]
                    # Exiba ou faça o que for necessário com as ROIs
                    cv2.imwrite('ROI A.png', roi_A)
                    cv2.imwrite('ROI B.png', roi_B)
                    cv2.imwrite('ROI C.png', roi_C)
                    cv2.imwrite('ROI D.png', roi_D)
                    cv2.imwrite('ROI E.png', roi_E)

                    # Função para detectar círculos pretos em uma ROI
                    def detectar_circulos_pretos(roi):
                        _, thresh = cv2.threshold(roi, 1, 255, cv2.THRESH_BINARY)
                        circulos = cv2.HoughCircles(thresh, cv2.HOUGH_GRADIENT, dp=1,minDist=8,param1=3,   param2=4,   minRadius=3,  maxRadius=10)
                        media_cor = np.mean(roi)
                        if(media_cor < 160):
                            return circulos
                        return None

                    # Detecte círculos nas ROIs
                    circulos_A = detectar_circulos_pretos(roi_A)
                    circulos_B = detectar_circulos_pretos(roi_B)
                    circulos_C = detectar_circulos_pretos(roi_C)
                    circulos_D = detectar_circulos_pretos(roi_D)
                    circulos_E = detectar_circulos_pretos(roi_E)

                    # Determine a resposta com base na detecção de círculos
                    if circulos_A is not None:
                        respostas.append("A")
                    elif circulos_B is not None:
                        respostas.append("B")
                    elif circulos_C is not None:
                        respostas.append("C")
                    elif circulos_D is not None:
                        respostas.append("D")
                    elif circulos_E is not None:
                        respostas.append("E")
                    else:
                        respostas.append("Nenhuma resposta detectada")
                    #cv2.imwrite(f'img combinada {j} - {i}.png', parte3)


            if prova == 1:
                gabarito = respostas_A
            elif prova == 2:
                gabarito = respostas_B
            elif prova == 3:
                gabarito = respostas_C
            elif prova == 4:
                gabarito = respostas_D
            elif prova == 5:
                gabarito = respostas_E

            questions = []
            questoes_certas = 0
            questoes_erradas = 0
            questoes_totais = 60
            erros = []
            for i in range(0,60):
                questions.append(i+1)
                if(respostas[i] == gabarito[i] or gabarito[i] == 'NULL'):
                    questoes_certas += 1
                    erros.append('ACERTOU')
                else: 
                    questoes_erradas += 1
                    erros.append(f'ALTERNATIVA COLOCADA: {respostas[i]}')

            nota_final = f"{questoes_certas}/{questoes_totais}"
            taxa_acerto = (questoes_certas/questoes_totais)*100
            nome_do_aluno = arquivo

            data = {
                'Questões': questions,
                'Questões Certas': [questoes_certas] * 60,  # Repetir o valor para todas as linhas
                'Questões Erradas': [questoes_erradas] * 60,
                'Erros': erros,
                'Nota Final': [nota_final] * 60,
                'Taxa de Acerto (%)': [taxa_acerto] * 60,
                'Nome do Aluno': [nome_do_aluno] * 60
            }

            df = pd.DataFrame(data)

            # caminho do arquivo Excel onde você deseja salvar as informações
            nome = 'notas'
            caminho_arquivo_excel = f'Leitor\{nome}\{arquivo}.xlsx'

            # Salve o DataFrame como uma planilha no arquivo Excel
            df.to_excel(caminho_arquivo_excel, index=False)

            print(f'Informações do aluno salvas em: {caminho_arquivo_excel}')
                
            # index += 1
            # cv2.imshow('image_resize', image_resize)
            # cv2.imshow('binary_frame', thresh)
            # cv2.imshow('image_copy', thresh)
        except:
            continue


cv2.waitKey(0)
cv2.destroyAllWindows()

entrou 1
0
entrou 2
0
0
entrou 3
0
0
entrou 4
entrou 5
255
5
A imagem é classificada como E
13 292
Informações do aluno salvas em: Leitor\notas\scan0000.jpg.xlsx
entrou 1
0
entrou 2
0
entrou 3
0
0
entrou 4
0
entrou 5
0
255
5
A imagem é classificada como E
12 276
Informações do aluno salvas em: Leitor\notas\scan0001.jpg.xlsx
entrou 1
255
1
A imagem é classificada como A
13 275
Informações do aluno salvas em: Leitor\notas\scan0002.jpg.xlsx
entrou 1
255
1
A imagem é classificada como A
0 600
Informações do aluno salvas em: Leitor\notas\scan0003.jpg.xlsx
entrou 1
255
1
A imagem é classificada como A
0 600
Informações do aluno salvas em: Leitor\notas\scan0004.jpg.xlsx
entrou 1
255
1
A imagem é classificada como A
11 272
Informações do aluno salvas em: Leitor\notas\scan0005.jpg.xlsx
entrou 1
255
1
A imagem é classificada como A
8 280
Informações do aluno salvas em: Leitor\notas\scan0006.jpg.xlsx
entrou 1
255
1
A imagem é classificada como A
12 274
Informações do aluno salvas em: Leitor\notas